2. Data Science

The algorithms I think are relevant to this project are logistic regression fitting, linear regression and power function fitting.
These three methods are all prediction methods for LTV.

In [170]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

The goal of this test is to implement a machine learning algorithm to forecast the LTV* of a user after 3 days in the application.

Based on the data we provide from one of our application, input.csv file, analyze and prepare the data to make it usable in an algorithm. Please explain all the steps in your analysis and preparation. Feel free to include some visualization of your analysis.
Now that you have analyzed and prepared the data, define at least three different algorithms that you think could be relevant to calculate the forecast of a user after 3 days in the application. Explain your choices.
Train one of them. Present the results.
*LTV = The life time value is the overall revenue generated by a user in an application. It includes in-app purchases and ad revenue.

In [171]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.000')
data = pd.read_csv('input.csv', parse_dates=['created_at', 'installed_at'], date_parser=dateparse)

Because of forecasting returns for the next three days, I calculated the time period for all the data

About the input.zip file

In [172]:
data.head()

,ab_test_version,activity_kind,ad_impressions_count,ad_mediation_platform,ad_revenue_network,ad_revenue_unit,adgroup_name,adid,app_version_short,att_status,...,idfa_android_id,idfa_gps_adid,installed_at,is_organic,network_name,os_name,reporting_revenue,session_count,sk_conversion_value,sub_details
0,38,session,0,NaN,NaN,NaN,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,NaN,NaN,2021-11-08 00:19:21,0,applovin,ios,0.0,4,0,NaN
1,38,event,0,NaN,NaN,NaN,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,NaN,NaN,2021-11-08 00:19:21,0,applovin,ios,0.0,4,0,NaN
2,38,event,0,NaN,NaN,NaN,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,NaN,NaN,2021-11-08 00:19:21,0,applovin,ios,0.0,4,0,NaN
3,29,session,0,NaN,NaN,NaN,NaN,220db1e74489be3928b976acb062a75f,2.7.4,2,...,NaN,NaN,2021-08-01 15:26:32,1,Organic,ios,0.0,13,0,NaN
4,29,event,0,NaN,NaN,NaN,NaN,220db1e74489be3928b976acb062a75f,2.7.4,2,...,NaN,NaN,2021-08-01 15:26:32,1,Organic,ios,0.0,13,0,NaN


In [173]:
data['activity_after_3d'] = data['created_at'] - data['installed_at']
data = data[data['activity_after_3d'] > pd.Timedelta(3, "d")]
data.drop(columns=['created_at', 'installed_at', 'activity_after_3d'], axis=1, inplace=True)

Then exclude the data in it

In [174]:
data['revenue_acc'] = data.groupby('adid')['reporting_revenue'].transform('sum')

In [175]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396718 entries, 0 to 2104307
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ab_test_version        396718 non-null  int64  
 1   activity_kind          396718 non-null  object 
 2   ad_impressions_count   396718 non-null  int64  
 3   ad_mediation_platform  91125 non-null   object 
 4   ad_revenue_network     90776 non-null   object 
 5   ad_revenue_unit        91125 non-null   object 
 6   adgroup_name           148170 non-null  object 
 7   adid                   396718 non-null  object 
 8   app_version_short      396718 non-null  object 
 9   att_status             396718 non-null  int64  
 10  campaign_name          148170 non-null  object 
 11  conversion_duration    396718 non-null  int64  
 12  country                390850 non-null  object 
 13  country_subdivision    286487 non-null  object 
 14  creative_name          142979 non-n

In [176]:
data.isnull().sum().sort_values(ascending=False)

sub_details              386935
ad_revenue_network       305942
ad_mediation_platform    305593
ad_revenue_unit          305593
creative_name            253739
adgroup_name             248548
campaign_name            248548
idfa_gps_adid            216004
idfa_android_id          216004
event_name               184796
country_subdivision      110231
device_manufacturer       97486
device_name               95864
country                    5868
network_name                123
sk_conversion_value           0
session_count                 0
reporting_revenue             0
os_name                       0
is_organic                    0
ab_test_version               0
device_type                   0
activity_kind                 0
conversion_duration           0
att_status                    0
app_version_short             0
adid                          0
ad_impressions_count          0
revenue_acc                   0
dtype: int64

In [177]:
cols_cat = ['sub_details', 'ad_revenue_network', 'ad_mediation_platform', 'ad_revenue_unit', 'creative_name', 'adgroup_name',
            'campaign_name', 'idfa_gps_adid', 'idfa_android_id', 'event_name', 'country_subdivision', 'device_name', 
            'device_manufacturer', 'country', 'network_name', 'device_type', 'app_version_short'
           ]
cols_num = ['session_count', 'ab_test_version', 'is_organic', 'ad_impressions_count', 'conversion_duration', 'att_status'
           ]

data[cols_cat] = data[cols_cat].fillna('')
data[cols_num] = data[cols_num].fillna(0)

Pad the corresponding data with nulls and 0s

In [178]:
data.head()

,ab_test_version,activity_kind,ad_impressions_count,ad_mediation_platform,ad_revenue_network,ad_revenue_unit,adgroup_name,adid,app_version_short,att_status,...,idfa_android_id,idfa_gps_adid,is_organic,network_name,os_name,reporting_revenue,session_count,sk_conversion_value,sub_details,revenue_acc
0,38,session,0,,,,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,,,0,applovin,ios,0.0,4,0,,0.00
1,38,event,0,,,,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,,,0,applovin,ios,0.0,4,0,,0.00
2,38,event,0,,,,969da0d8a7b4aff20da9d6482dd4cfbd,1ccaf44b0d403de7c52f887db8aa98b4,3.0,2,...,,,0,applovin,ios,0.0,4,0,,0.00
3,29,session,0,,,,,220db1e74489be3928b976acb062a75f,2.7.4,2,...,,,1,Organic,ios,0.0,13,0,,0.05
4,29,event,0,,,,,220db1e74489be3928b976acb062a75f,2.7.4,2,...,,,1,Organic,ios,0.0,13,0,,0.05


Some data are invalid, such as operating system, version, etc.

In [179]:
data['os_name'].value_counts()

ios    396718
Name: os_name, dtype: int64

In [180]:
cols_to_remove = ['idfa_gps_adid', 'idfa_android_id', 'device_manufacturer', 'os_name', 'sub_details','ad_revenue_network',              
                  'sk_conversion_value', 'session_count', 'conversion_duration','ad_revenue_unit','creative_name',
                  'reporting_revenue', 'activity_kind', 'adid'
                 ]
data.drop(columns=cols_to_remove, axis=1, inplace=True)
print(data.shape)

(396718, 15)


In [181]:
le = preprocessing.LabelEncoder()
data[cols_cat] = le.fit_transform(cols_cat)

In [182]:
label = ['revenue_acc']
all_features = [col for col in data.columns if col not in label]

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(data[all_features])
y = data[label].values

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [184]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [185]:
preds_revenues = linreg.predict(X_test)
pred_rmse = mean_squared_error(y_test, preds_revenues)
print("RMSE error on validset is :", round(pred_rmse, 3))

RMSE error on validset is : 12.614
